In [4]:
# Asegúrate de que db_connection.py esté en la ruta correcta y contenga get_connection
from db_connection import get_connection

# 1. Reconectar (o asegurar que la conexión esté activa)
try:
    conexion = get_connection() # Obtener una nueva conexión o reutilizar si get_connection maneja eso
    cursor = conexion.cursor()

    # 2. Ejecutar la consulta para describir la tabla
    cursor.execute("DESCRIBE accounting_account_balances;")
    columns_info = cursor.fetchall()

    print("Columnas de la tabla 'accounting_account_balances':")
    for column in columns_info:
        print(f"  - {column[0]} ({column[1]})")

except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    # 3. Cerrar el cursor y la conexión de forma segura
    if 'cursor' in locals() and cursor is not None:
        cursor.close()
    if 'conexion' in locals() and conexion.is_connected():
        conexion.close()
    print("Conexión y cursor cerrados.")

✅ Conexión exitosa a la base de datos
Columnas de la tabla 'accounting_account_balances':
  - id (bigint unsigned)
  - code (double(16,15))
  - accounting_id (bigint unsigned)
  - name (varchar(255))
  - initial_balance (double(18,6))
  - final_balance (double(18,6))
  - debit_movement (double(18,6))
  - credit_movement (double(18,6))
  - third_party_type_id (varchar(50))
  - third_party_id (bigint unsigned)
  - currency_id (varchar(255))
  - year (int)
  - month (int)
  - created_at (timestamp)
  - updated_at (timestamp)
Conexión y cursor cerrados.
